In [1]:
#importing CSV File and Reading it
import pandas as pd
import csv
import pyspark
from pyspark import SparkContext, SparkConf
import numpy as np
import tensorflow as tf
import plotly.plotly as py
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr, zscore

/Users/abhinavjain/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#Starting the Spark Session
conf=SparkConf().setAppName("CSE545 Project").set("spark.driver.memory", "12g").set("spark.executor.memory", "6g").set("spark.driver.maxResultSize", "6g")
sc=SparkContext(conf=conf)

In [3]:
#Loading WDI Dataset
WDI_rdd1 = sc.textFile("WDI_GDP_Growth.csv").map(lambda line: line.split(",")).filter(lambda line: len(line)>1)

In [4]:
print(WDI_rdd1.collect())

[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', 'Unnamed: 61'], ['Arab World', 'ARB', 'GDP growth (annual %)', 'NY.GDP.MKTP.KD.ZG', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '15.9031516079944', '8.50550096656832', '-0.7984761869222149', '11.605057353708', '9.145067518629618', '2.68708383254845', '-8.9925725353927', '-6.573568995782581', '1.06640451994264', '-2.16460950525548', '4.46288865462701', '-0.626386706730514', '5.58877530570663', '2.12100068254981', '13.141355032793902', '1.5296157817614

In [5]:
#Filtering required data
WDI_rdd2 = WDI_rdd1.map(lambda x: [x[0],x[29:60]])

print(WDI_rdd2.collect())

[['Country Name', ['1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']], ['Arab World', ['-2.16460950525548', '4.46288865462701', '-0.626386706730514', '5.58877530570663', '2.12100068254981', '13.141355032793902', '1.52961578176148', '4.79539500272912', '3.20093873984074', '3.209841598949', '2.7552678606214602', '4.478689158267731', '4.18729746437975', '5.1001984195861505', '1.80769841826131', '5.37168948140877', '1.61383675424491', '0.5844055138085339', '5.3192667581860595', '9.335660113583051', '5.71629757147758', '6.495442352866861', '4.57174871901114', '5.81595309929904', '0.43183141309322104', '4.77378908129691', '3.57043337524128', '5.16139739851323', '3.64693465504243', '2.9113462019072203', '3.4848081657532104']], ['Caribbean small states', ['-0.7137576199960729', '0.8062641859168309',

In [6]:
# headers = WDI_rdd2.collect()[0]
# WDI_rdd2=WDI_rdd2.filter(lambda x: x[0]!='Country Name')

# df0 = pd.DataFrame(WDI_rdd2.collect(), columns=headers)

In [7]:
#Converting String Values to Integer Values
def conv_x(x):
    growth=[]
    count=0
    temp=[]
    if x[0] != "Country Name":
        for num in x[1]:
            if num == '':
                temp.append(0)
                count+=1
                if(count==8):
                    growth.append(temp)
                    temp=[]
                    count=0
            else:
                temp.append(round(float(num),2))
                count+=1
                if(count==8):
                    growth.append(temp)
                    temp=[]
                    count=0
        growth.append(temp)
    else:
        growth=x[1]
    return [x[0],growth]

In [8]:
#Calculating growth over 8 years
def calc_growth8(x):
    wdi=[]
    if x[0] != "Country Name":
        for l in x[1]:
            gr=0
            for i in range(len(l)):
                gr=round(float(l[i] + gr + (l[i]*gr/100)),2)
            wdi.append(gr)
    else:
        wdi=x[1]
    return [x[0],wdi]

#### Call to the conversion function
WDI_rdd3=WDI_rdd2.map(lambda x: conv_x(x)).map(lambda x: calc_growth8(x))
print(WDI_rdd3.collect())

In [75]:
#Transforming rdd to tensors for applying Machine Learning Models
X_wdi=WDI_rdd3.filter(lambda x: x[0]!="Country Name").map(lambda x: x[1])
X_wdi=np.array(X_wdi.collect())
print(X_wdi)

[[ 31.84  34.34  46.63  26.49]
 [ 10.89  28.57  35.85   2.36]
 [-11.33  31.25  45.23  10.02]
 ..., 
 [  6.29  53.73  39.12   8.41]
 [  8.08  19.92  70.22  54.9 ]
 [ 28.37  23.95 -47.84  82.01]]


In [76]:
def calc_beta(betas, X_test, y_test):
    y_pred = np.matmul(X_test, betas)[:,0]
    print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))

In [97]:
def RidgeRegression(X_wdi,penalty_value = 1.0, learning_rate = 0.000000001, n_epochs = 100):

    #Dividing into training and test data
    offset = int(int(X_wdi.shape[0]) * 0.8)
    X_wdi_tf_test, Y_wdi_tf_test = X_wdi[offset:,:3], X_wdi[offset:,3:]
    X_wdi_tf, Y_wdi_tf = X_wdi[:offset,:3], X_wdi[:offset,3:]

    # Conversion to tensors
    X_wdi_tf = tf.constant(X_wdi_tf, dtype=tf.float32, name="WDI_X")
    Y_wdi_tf = tf.constant(Y_wdi_tf.reshape(-1,1), dtype=tf.float32, name="WDI_Y")
    Xt_wdi_tf = tf.transpose(X_wdi_tf)
    penalty = tf.constant(1.0, dtype=tf.float32, name="penalty")
    I = tf.constant(np.identity(int(X_wdi_tf.shape[1])), dtype=tf.float32, name="I")
    beta = tf.Variable(tf.random_uniform([int(X_wdi_tf.shape[1]), 1], -1., 1.), name = "beta")
    y_pred = tf.matmul(X_wdi_tf, beta, name="predictions")
    penalizedCost = tf.reduce_sum(tf.square(Y_wdi_tf - y_pred)) + penalty * tf.reduce_sum(tf.square(beta))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(penalizedCost)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(n_epochs):
            if epoch %10 == 0: #print debugging output
                print("Epoch", epoch, "; penalizedCost =", penalizedCost.eval())
            sess.run(training_op)
        #done training, get final beta: 
        best_beta = beta.eval()
    print(best_beta)
    calc_beta(best_beta, X_wdi_tf_test, Y_wdi_tf_test)
    return best_beta

best_beta_val = RidgeRegression(X_wdi,.1)

Epoch 0 ; penalizedCost = 779057.0
Epoch 10 ; penalizedCost = 727944.0
Epoch 20 ; penalizedCost = 680941.0
Epoch 30 ; penalizedCost = 637714.0
Epoch 40 ; penalizedCost = 597958.0
Epoch 50 ; penalizedCost = 561393.0
Epoch 60 ; penalizedCost = 527759.0
Epoch 70 ; penalizedCost = 496819.0
Epoch 80 ; penalizedCost = 468356.0
Epoch 90 ; penalizedCost = 442169.0
[[ 0.16687308]
 [ 0.3216235 ]
 [ 0.40848088]]
Mean Absolute Error: 20.4639764969


In [98]:
sess = tf.InteractiveSession()
best_beta_val = np.array(best_beta_val, dtype=np.float64)
WDI_2024 = tf.matmul(X_wdi[:,1:], best_beta_val, name="predictions")
s = WDI_2024.eval()
print(s)

#close the session to release resources
sess.close()

[[  31.54838423]
 [  17.26178147]
 [  23.85479337]
 [  33.19554156]
 [  33.45192384]
 [  74.23588995]
 [  74.23422122]
 [   8.75151528]
 [  12.2576134 ]
 [  25.2157266 ]
 [  25.42449679]
 [  10.52920817]
 [  33.63108192]
 [  38.79888758]
 [  13.6491348 ]
 [  41.75593296]
 [  41.73599061]
 [  41.83804025]
 [  41.301889  ]
 [  41.50448185]
 [  46.88106498]
 [  21.49146956]
 [  21.40045947]
 [  21.87099824]
 [  44.47189711]
 [  42.23131367]
 [  36.96866345]
 [  44.61069294]
 [  29.48815613]
 [  26.76202293]
 [  26.71855789]
 [  42.20210425]
 [  16.12155674]
 [   0.        ]
 [  13.3743217 ]
 [  37.72257151]
 [  17.58794455]
 [  12.58938949]
 [  45.07105896]
 [  33.10889579]
 [  55.5962247 ]
 [  55.5962247 ]
 [  36.69587191]
 [  36.700446  ]
 [  36.69587191]
 [  41.51804635]
 [  20.99171448]
 [  43.70690706]
 [  38.54725287]
 [  25.42764712]
 [  -4.73111449]
 [  11.96126422]
 [  56.90846486]
 [  15.9276944 ]
 [  19.82411682]
 [  55.41182881]
 [   1.01826919]
 [  23.7113164 ]
 [  11.0707753

In [99]:
X_wdi_df = pd.DataFrame(X_wdi)
X_wdi_df = X_wdi_df.rename(index=str, columns={0: "1992", 1: "2000",2: "2008",3: "2015"})
X_wdi_df['2024'] = s

In [100]:
X_wdi_2024 = np.array(X_wdi_df)
sess = tf.InteractiveSession()
best_beta_val = np.array(best_beta_val, dtype=np.float64)
WDI_2032 = tf.matmul(X_wdi_2024[:,2:], best_beta_val, name="predictions")
s = WDI_2032.eval()
print(s)

#close the session to release resources
sess.close()

[[  2.91880103e+01]
 [  1.37925392e+01]
 [  2.05145641e+01]
 [  3.18633028e+01]
 [  3.19717102e+01]
 [  6.91554415e+01]
 [  6.91547598e+01]
 [  6.49120540e+00]
 [  1.03643927e+01]
 [  2.56394745e+01]
 [  2.54687534e+01]
 [  8.19532194e+00]
 [  2.88337074e+01]
 [  3.82734409e+01]
 [  1.12828244e+01]
 [  4.00420072e+01]
 [  4.00785247e+01]
 [  4.14399994e+01]
 [  3.99497183e+01]
 [  4.06661992e+01]
 [  4.46389300e+01]
 [  1.93280597e+01]
 [  1.93464084e+01]
 [  1.97334070e+01]
 [  4.27061853e+01]
 [  4.06747754e+01]
 [  3.70690190e+01]
 [  4.36548568e+01]
 [  2.70564444e+01]
 [  2.33567472e+01]
 [  2.32996988e+01]
 [  4.06124752e+01]
 [  1.30480918e+01]
 [  0.00000000e+00]
 [  1.10282699e+01]
 [  3.88387712e+01]
 [  1.67525115e+01]
 [  1.02713875e+01]
 [  4.28252633e+01]
 [  3.35139880e+01]
 [  5.34017207e+01]
 [  5.34017207e+01]
 [  3.54938835e+01]
 [  3.54976631e+01]
 [  3.54938835e+01]
 [  3.97479668e+01]
 [  1.89716592e+01]
 [  4.78147603e+01]
 [  3.13974162e+01]
 [  2.45324392e+01]


In [101]:
X_wdi_2024 = pd.DataFrame(X_wdi_2024)
X_wdi_2024 = X_wdi_2024.rename(index=str, columns={0: "1992", 1: "2000",2: "2008",3: "2015",4: "2024"})
X_wdi_2024['2032'] = s

In [103]:
X_wdi_2032 = np.array(X_wdi_2024)
sess = tf.InteractiveSession()
best_beta_val = np.array(best_beta_val, dtype=np.float64)
WDI_2040 = tf.matmul(X_wdi_2032[:,3:], best_beta_val, name="predictions")
s = WDI_2040.eval()
print(s)

#close the session to release resources
sess.close()

X_wdi_2032 = pd.DataFrame(X_wdi_2032)
X_wdi_2032 = X_wdi_2032.rename(index=str, columns={0: "1992", 1: "2000",2: "2008",3: "2015",4: "2024",5:"2032"})
X_wdi_2032['2040'] = s

[[  26.48991405]
 [  11.57960371]
 [  17.72413776]
 [  29.39907586]
 [  29.63595303]
 [  63.47038371]
 [  63.46956857]
 [   5.63476814]
 [   9.01536448]
 [  21.12802005]
 [  21.29563965]
 [   7.2230113 ]
 [  26.90322764]
 [  35.29648929]
 [  10.3720661 ]
 [  36.30414651]
 [  36.34101772]
 [  37.36885188]
 [  36.53759977]
 [  36.92379557]
 [  40.26246583]
 [  17.47894269]
 [  17.47385411]
 [  17.85836485]
 [  38.4795278 ]
 [  36.85735597]
 [  34.55461525]
 [  39.95630745]
 [  24.4292734 ]
 [  20.72626944]
 [  20.65561227]
 [  36.78079483]
 [  12.28715978]
 [   0.        ]
 [  10.1630118 ]
 [  33.86628749]
 [  15.90398794]
 [   9.40447693]
 [  38.75091057]
 [  29.20944776]
 [  49.81882454]
 [  49.81882454]
 [  31.7592463 ]
 [  31.75892387]
 [  31.7592463 ]
 [  35.87724185]
 [  17.40791795]
 [  44.6706255 ]
 [  28.17826913]
 [  22.19744043]
 [  -5.10878458]
 [   4.18837262]
 [  46.32746104]
 [   7.7972031 ]
 [  15.46293453]
 [  39.49030136]
 [  -1.28382089]
 [  19.2208245 ]
 [   7.6572978

In [110]:
X_wdi_2032 = X_wdi_2032.round(2)

In [111]:
X_wdi_2032

,1992,2000,2008,2015,2024,2032,2040
0,31.84,34.34,46.63,26.49,31.55,29.19,26.49
1,10.89,28.57,35.85,2.36,17.26,13.79,11.58
2,-11.33,31.25,45.23,10.02,23.85,20.51,17.72
3,31.07,30.85,43.77,34.20,33.20,31.86,29.40
4,51.04,33.18,42.52,34.86,33.45,31.97,29.64
5,82.47,82.50,101.66,67.99,74.24,69.16,63.47
6,82.47,82.49,101.66,67.99,74.23,69.15,63.47
7,25.94,20.04,15.53,1.01,8.75,6.49,5.63
8,19.94,17.95,22.41,5.03,12.26,10.36,9.02
9,-16.31,-6.74,62.53,15.25,25.22,25.64,21.13
